## Table of Contents - **Data Preparation**

  **STEP 1:** Clean all numerical features and the target variable price so that they can be used in training algorithms (as a regular feature).
  
  **STEP 2:** Clean all features that contains multiple items of information, e.g. creating email, phone, work_email, etc. from feature host_verifications.
  
  **STEP 3:** Impute missing values for all features in both training and test datasets. 
  
  **STEP 4:** Encode all categorical variables appropriately.
  
  **STEP 5:** Create dummy variables on amenities feature. 

  **STEP 6:** Perform exploratory data analysis to measure the relationship between the features and the target.

**STEP 1: Clean all numerical features and the target variable price so that they can be used in training algorithms (as a regular feature).**

In [ ]:
# remove % sign for host_response_rate and host_acceptance_rate
df['host_response_rate'] = df['host_response_rate'].str.replace('%', '').astype(float) / 100
df['host_acceptance_rate'] = df['host_acceptance_rate'].str.replace('%', '').astype(float) / 100

# remove $ sign for price target variable
df['price'] = df['price'].str.replace('[\$,]', '', regex=True).astype(float)

# convert the string half/Half to a number 0.5
df['bathrooms'] = df['bathrooms'].str.replace('half', '0.5 half')
df['bathrooms'] = df['bathrooms'].str.replace('Half', '0.5 half')
df['Bath Type'] = df['bathrooms'].astype(str).apply(lambda x: 'Shared' if 'hared' in x else ('Private' if 'rivate' in x else 'Normal'))
df['bathrooms'] = df['bathrooms'].str.extract('(\d+\.?\d*)').astype(float)

Explanation: 
1. when we looked the data, we observe that the variable host_response_rate, host_acceptance_rate have the percentage sign, and price has the dollar sign. We extract numerical values by removing the percentage sign and dollar sign.
2. Then we observe for bathroom feature, some value recorded as half/Half, so we convert the text to a number 0.5 and eliminate text for the bathroom column.

**STEP 2: Clean all features that contains multiple items of information, e.g. creating email, phone, work_email, etc. from feature host_verifications.**

In [ ]:
# creating 4 new features from host_verification feature
df['email_verified'] = df['host_verifications'].apply(lambda x: 'email' in x).astype(int)
df['phone_verified'] = df['host_verifications'].apply(lambda x: 'phone' in x).astype(int)
df['work_email_verified'] = df['host_verifications'].apply(lambda x: 'work_email' in x).astype(int)
df['num_verifications'] = df['host_verifications'].apply(lambda x: len(x.split(',')) if x else 0)
# print(df['host_verifications'])
# print(df.head())

In [ ]:
# split amenities into separate values and create a new row for each value
amenities = df['amenities'].str.split(', ')
amenities = amenities.explode()

# count the frequency of each value
value_counts = amenities.value_counts()
# print the top 5 most frequent items
print(value_counts.head(5))

df['has_Kitchen'] = df['amenities'].str.contains('Kitchen', regex=False).astype(int)
df['has_Smoke alarm'] = df['amenities'].str.contains('Smoke alarm', regex=False).astype(int)
df['Hangers'] = df['amenities'].str.contains('Hangers', regex=False).astype(int)
df['Wifi'] = df['amenities'].str.contains('Wifi', regex=False).astype(int)
df['Iron'] = df['amenities'].str.contains('Iron', regex=False).astype(int)
# print(df)

Explanation: 

When we observe the dataset, we observe the **host_verification and amenities** holding mulitple items. For the host_verification, I divided the 3 verifications to different features as dummy variable. As for the amenities feature has a lot more items, so I select the top 5 most frequent items in the amenities and divide them into different features as dummy variables.

**STEP 3: Impute missing values for all features in both training and test datasets.**

**Checking missing values**

In [ ]:
print('No. of null value in each variable\n',df.loc[:, 'ID':'reviews_per_month'].isnull().sum())

**Addressing missing values**
- **Handling location**

('neighbourhood', 'neighbourhood_cleansed')

In [ ]:
#clean suburb to matches neighbourhood cleansed 
df['municipality'] = df['municipality'].str.replace('City of ', '')
df['municipality'] = df['municipality'].str.replace('Shire of ', '')
df['neighbourhood'] = df['suburb'] #makes the neighbourhood  column = suburb
df['neighbourhood_cleansed'].fillna(df['municipality'], inplace=True) #use the municipality to replace na values for neighbourhood_cleansed
# print('No. of null value in each variable\n',df.loc[:, 'ID':'reviews_per_month'].isnull().sum())

- **Handling text variables**

('name', 'description', 'neighborhood_overview', 'host_about')

In [ ]:
# Fill the missing value in name, description, neighborhood_overview, and host_about with 'no information'
df['name'] = df['name'].fillna('no information')
df['description'] = df['description'].fillna('no information')
df['neighborhood_overview'] = df['neighborhood_overview'].fillna('no information')
df['host_about'] = df['host_about'].fillna('no information')
# print('No. of null value in name variable\n',df['name'].isnull().sum())
# print('No. of null value in description variable\n',df['description'].isnull().sum())
# print('No. of null value in neighborhood_overview variable\n',df['neighborhood_overview'].isnull().sum())
# print('No. of null value in host_about variable\n',df['host_about'].isnull().sum())

- **Fill missing values in categorical variables**

('host_response_time', 'host_is_superhost','host_location', 'host_neighbourhood', 'property_type', 'room_type')

In [ ]:
# Divide the training dataset and test dataset
df_train_clean = df.iloc[:7000]
# print(df_train_clean)
df_test_clean = df.iloc[7000:]
# print(df_test_clean)


# Fill in the missing values in categorical variables with the most frequent values
from scipy.stats import mode
columns_to_process = ['host_response_time', 'host_is_superhost','host_location', 'host_neighbourhood', 'property_type', 'room_type']

modes = df_train_clean[columns_to_process].mode(axis=0)
for column in columns_to_process:
    df[column].fillna(modes[column][0], inplace = True)

- **Fill missing values in numerical variables**

('host_response_rate', 'host_acceptance_rate', 'bathrooms', 'bedrooms', 'beds', 'minimum_minimum_nights', 'maximum_maximum_nights', 'availability_365', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month')

In [ ]:
# Fill in the missing values in numerical variables with the mean value
columns_to_fill_mean = ['host_response_rate', 'host_acceptance_rate', 'bathrooms', 'bedrooms', 'beds', 'minimum_minimum_nights', 'maximum_maximum_nights', 'availability_365', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month']

for column in columns_to_fill_mean:
    df[column].fillna(df_train_clean[column].mean(), inplace = True)   
# print('No. of null value in each variable\n',df.loc[:, 'ID':'reviews_per_month'].isnull().sum())

- **FIll missing values for date type** 

('first_review', 'last_review')

In [ ]:
# convert date column to datetime type
df['first_review'] = pd.to_datetime(df['first_review'])
df['last_review'] = pd.to_datetime(df['last_review'])
# impute missing values with the following date value
df['first_review'] = df['first_review'].fillna(method='ffill')
df['last_review'] = df['last_review'].fillna(method='ffill')
# print(df['first_review'])

# print('No. of null value in date variable\n',df['first_review'].isnull().sum())

# print('No. of null value in each variable\n',df.loc[:, 'ID':'reviews_per_month'].isnull().sum())

Explanation: 

##### Addressing missing values
- **Handling location** - Geopy

('neighbourhood', 'neighbourhood_cleansed')
- **Handling text variables** - fill in 'no informaiton'

('name', 'description', 'neighborhood_overview', 'host_about'
- **Fill missing values in categorical variables** - using most frequent values in training dataset 

('host_response_time', 'host_is_superhost','host_location', 'host_neighbourhood', 'property_type', 'room_type')
- **Fill missing values in numerical variables** - using mean values in training dataset

('host_response_rate', 'host_acceptance_rate', 'bathrooms', 'bedrooms', 'beds', 'minimum_minimum_nights', 'maximum_maximum_nights', 'availability_365', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'reviews_per_month')
- **FIll missing values for date type** - same value as the followed non-null value

('first_review', 'last_review')

**STEP 4: Encode all categorical variables appropriately.**

Where a categorical feature contains more than 5 unique values, map the feature into 5 most frequent values + 'other' and then encode appropriately. For instance, you could group then map `property_type` into 5 most frequent property types + 'other'  
(2 marks)

In [ ]:
##Checking: 
print(df[['host_location', 'host_verifications', 'neighbourhood', 'property_type', 'host_neighbourhood','neighbourhood_cleansed', 'room_type', 'host_response_time']].nunique())

#### Variables need to encode: 
**property_type, host_location, neighbourhood, host_neighbourhood, and neighbourhood_cleansed**
- One-hot encoding
Since these are nominal categorical variables merge all encoded variables to the original dataset. 

In [ ]:
most_frequent = 5  # Number of most frequent values to keep

# Get value counts of the property_type feature
property_type_counts = df['property_type'].value_counts()

# print(property_type_counts)

# Select the top 5 values and 'other'
top_values_property = property_type_counts.head(most_frequent).index.tolist()
top_values_property.append('other')

# print(top_values_property)

df['property_type_mapped'] = df['property_type'].map(lambda x: x if x in top_values_property else 'other')

# print(df['property_type_mapped'])

# Perform one-hot encoding
encoded_property = pd.get_dummies(df['property_type_mapped'], prefix='property_type')
# print(encoded_property)



# Get value counts of the host_location feature
host_location_counts = df['host_location'].value_counts()

# print(host_location_counts)

# Select the top N values and 'other'
top_values_host = host_location_counts.head(most_frequent).index.tolist()
top_values_host.append('other')

# print(top_values_host)

df['host_location_mapped'] = df['host_location'].map(lambda x: x if x in top_values_host else 'other')

# print(df['host_location_mapped'])

# Perform one-hot encoding
encoded_host = pd.get_dummies(df['host_location_mapped'], prefix='host_location')
# print(encoded_host)



# Get value counts of the neighborhood feature
nbr_counts = df['neighbourhood'].value_counts()

# print(nbr_counts)

# Select the top N values and 'other'
top_values_nbr = nbr_counts.head(most_frequent).index.tolist()
top_values_nbr.append('other')

# print(top_values_host)

df['nbr_mapped'] = df['neighbourhood'].map(lambda x: x if x in top_values_nbr else 'other')

# print(df['host_location_mapped'])

# Perform one-hot encoding
encoded_nbr = pd.get_dummies(df['nbr_mapped'], prefix='neighbourhood')
# print(encoded_nbr)



# Get value counts of the host_neighborhood feature
host_nbr_counts = df['host_neighbourhood'].value_counts()

# print(host_nbr_counts)

# Select the top N values and 'other'
top_values_host_nbr = host_nbr_counts.head(most_frequent).index.tolist()
top_values_host_nbr.append('other')

# print(top_values_host_nbr)

df['host_nbr_mapped'] = df['host_neighbourhood'].map(lambda x: x if x in top_values_host_nbr else 'other')

# print(df['host_nbr_mapped'])

# Perform one-hot encoding
encoded_host_nbr = pd.get_dummies(df['host_nbr_mapped'], prefix='host_neighbourhood')
# print(encoded_host_nbr)



# Get value counts of the neighbourhood_cleansed feature
nbr_cln_counts = df['neighbourhood_cleansed'].value_counts()

# print(nbr_cln_counts)

# Select the top N values and 'other'
top_values_nbr_cln = nbr_cln_counts.head(most_frequent).index.tolist()
top_values_nbr_cln.append('other')

# print(top_values_nbr_cln)

df['nbr_cln_mapped'] = df['neighbourhood_cleansed'].map(lambda x: x if x in top_values_nbr_cln else 'other')

# print(df['nbr_cln_mapped'])

# Perform one-hot encoding
encoded_nbr_cln = pd.get_dummies(df['nbr_cln_mapped'], prefix='neighbourhood_cleansed')
# print(encoded_nbr_cln)


# merge all encoded variables to the original dataset
df = pd.concat([df, encoded_property, encoded_host, encoded_nbr, encoded_host_nbr, encoded_nbr_cln], axis = 1)

**STEP 5: Create dummy variables on amenities feature**

In [ ]:
Intuitively, we think the balcony, private_balcony, courtyard_view, vineyard_view, TV and AC are affecting the price.

Also, the bathroom types can influence the price. Three bathroom types defined previously (normal bathroom, private bathroom, public bathroom)

In [ ]:
# count how many items for each Airbnb service
amenities_count = df['amenities'].str.split(', ').apply(len)
df['amenities_count'] = amenities_count
print(amenities_count)
# print(df.head())

In [ ]:
# select some variable items from amenities in each Airbnb house and make dummy variables
df['has_balcony'] = df['amenities'].str.contains('patio or balcony|Patio or balcony', case=False, regex=True).astype(int)
df['has_private_balcony'] = df['amenities'].str.contains('Private patio or balcony', case=False, regex=True).astype(int)
df['has_courtyard_view'] = df['amenities'].str.contains('Courtyard view', case=False, regex=True).astype(int)
df['has_vineyard_view'] = df['amenities'].str.contains('Vineyard view', case=False, regex=True).astype(int)
df['has_TV'] = df['amenities'].str.contains('TV', case=False, regex=False).astype(int)
df['has_AC'] = df['amenities'].str.contains('AC|Air conditioning', case=False, regex=True).astype(int)
# print(df)

In [ ]:
# encode each bath type (normal bath, private bath, shared bath)
pd.get_dummies(df['Bath Type'], prefix='Bath Type')
encoded_bath_type = pd.get_dummies(df['Bath Type'], prefix='Bath Type')
print(encoded_bath_type)

df = pd.concat([df, encoded_bath_type], axis = 1)

**STEP 6: Perform exploratory data analysis to measure the relationship between the features and the target**

In [ ]:
# print(df.dtypes.unique())
df_clean = df.select_dtypes(include=['int64', 'float64', 'int32'])
# print(df_clean)


# Calculate the correlation matrix
correlation_matrix = df_clean.corr()

# Filter features based on correlation threshold

# Print the strongly correlated features
print(abs(correlation_matrix['price']).sort_values(ascending=False).head(25))

In [ ]:
# Filter features based on correlation threshold
correlation_threshold = 0.015  # Adjust the threshold as per your requirement
strong_correlated_features = correlation_matrix[abs(correlation_matrix['price']) > correlation_threshold]['price']

# Print the strongly correlated features
print(abs(strong_correlated_features).sort_values(ascending=False))

# Select the strong correlated features and use them as input for traning and testing
df_final = df_clean[['bedrooms','accommodates', 'minimum_minimum_nights', 'bathrooms', 'beds', 'availability_365', 'minimum_nights_avg_ntm', 'availability_30', 'reviews_per_month', 'maximum_nights', 'minimum_nights', 'email_verified', 'maximum_minimum_nights', 'has_TV', 'number_of_reviews_ltm', 'num_verifications', 'number_of_reviews','number_of_reviews_l30d','price']]

df_train_final = df_final.iloc[:7000]
# print(df_train_final)
df_test_final = df_final.iloc[7000:]
# print(df_test_final)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Calculate correlations between selected features and the target
correlation_matrix_plot = df_final.corr()
plt.figure(figsize = (15,8))
sns.heatmap(correlation_matrix_plot, annot=True, cmap='coolwarm', vmin = -0.1, vmax = 0.1)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

import seaborn as sns

sns.pairplot(df_final)

plt.show()

We tried to select all numerical variables which has int64, float32, int32 and uint8. We iterate through data cleaning, feature engineering, and exploration steps to refine the dataset. After tried different errors, We found that the variables with int64, float32 and int32 with variables correlation greater than 0.015 gave us the best result. The resulting dataset will be used for training and evaluating the predictive models.

Selected features as follows: ('bedrooms','accommodates', 'minimum_minimum_nights', 'bathrooms', 'beds', 'availability_365', 'minimum_nights_avg_ntm', 'availability_30', 'reviews_per_month', 'maximum_nights', 'minimum_nights', 'email_verified', 'maximum_minimum_nights', 'has_TV', 'number_of_reviews_ltm', 'num_verifications', 'number_of_reviews','number_of_reviews_l30d')

Those features do not present a strong linear relationship